In [1]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import seaborn as sns 
import statsmodels.stats.proportion as proportion
from scipy.stats import ttest_ind,mannwhitneyu,shapiro,norm
from statsmodels.stats.weightstats import ztest
from tqdm import tqdm
import timeit
from scipy import stats
import math
from datetime import date, datetime, timedelta
import time
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import warnings
warnings.filterwarnings("ignore")
import clickhouse_connect  



from credential import postgres_secret,clickhouse_dwh_secret

def get_engine(user):
    if user == postgres_secret['user']:
        db_name = postgres_secret['db_name']
        password = postgres_secret['password']
        host = postgres_secret['host']
        engine = create_engine(f'postgresql://{user}:{password}@{host}:6432/{db_name}')
    elif user == clickhouse_dwh_secret['user']:
            db_name = clickhouse_dwh_secret['db_name'] 
            password = clickhouse_dwh_secret['password']
            host = clickhouse_dwh_secret['host']
            engine = create_engine(f'clickhouse://{user}:{password}@{host}:8123/{db_name}')
    return engine
    
connection_clickhouse = clickhouse_connect.get_client(
    host = clickhouse_dwh_secret['host'],
    port= '8123',
    username = clickhouse_dwh_secret['user'],
    password = clickhouse_dwh_secret['password'],
    database='datamarts'
    )

    
def execute(SQL, user):
    start_time = time.time()  # запоминаем время начала выполнения функции
    engine = get_engine(user)
    Session = sessionmaker(bind=engine)  # sessions factory ()
    with Session() as session: # open session
        result = session.execute(text(SQL))
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
        
    end_time = time.time()  # запоминаем время окончания выполнения функции
    execution_time = round(end_time - start_time,4) # вычисляем время выполнения   
    
    print(f"Время выполнения функции: {execution_time} секунд")
    print()
    return df

In [ ]:
query = f''' 
truncate table datamarts.financial_activity_local on cluster 'viasat_cluster' '''
df = execute(query,user = 'kmekhtiev')

query = f''' 
insert into datamarts.financial_activity_local
WITH money_table AS (
	SELECT 
		subscription_id,
		platform,
		toFloat32(i.price_cents) AS price_cents,
		i.price_currency AS price_currency,
		round(multiIf(
				price_currency = 'USD', price_cents * 90,			-- исправить хардкод
	    		price_currency = 'AMD', price_cents / 4.2,			-- исправить хардкод
				price_cents)
				, 2) as rubles,
		s.user_id AS user_id,
		s.created_at AS sub_create_ts,
		paid_at
	from product_x.invoices i 
	left join product_x.subscriptions s
		on s.id = i.subscription_id 
	LEFT JOIN product_x.users u ON u.id = s.user_id
	where 
		toDate(paid_at) >= '2023-09-01' 
        AND i.state in ('success')
        AND s.state in ('normal_period','trial','canceled','grace_period')
        AND price_cents > 100
        AND u.user_type = 'regular'
		AND 
		    (
		        (u.email NOT ILIKE '%%@test%%' AND u.email NOT ILIKE '%%@viasat%%')
		    OR
		    (u.email IS NULL AND u.phone_number IS NOT NULL)
		)
		AND u.vipplay = FALSE
),

sums AS (
	SELECT DISTINCT
		user_id,
		toDate(paid_at) as paid_date,
		toStartOfWeek(paid_at, 1) as paid_week,
		toStartOfMonth(paid_at) as paid_month,
		platform,
		sum(rubles/100) over(PARTITION BY user_id, paid_date) AS total_sum_by_day,
		sum(rubles/100) over(PARTITION BY user_id, paid_week) AS total_sum_by_week,
		sum(rubles/100) over(PARTITION BY user_id, paid_month) AS total_sum_by_month,
		
		sum(rubles/100) over(PARTITION BY user_id, platform, paid_date) AS platform_sum_by_day,
		sum(rubles/100) over(PARTITION BY user_id, platform, paid_week) AS platform_sum_by_week,
		sum(rubles/100) over(PARTITION BY user_id, platform, paid_month) AS platform_sum_by_month
	from money_table
	order by user_id, paid_date
),

users_info AS (
	SELECT DISTINCT
		user_id,
		multiIf(
			device='web' and device_os in ('iOS', 'Android'), 'web_mobile',
			device='web' and device_os not in ('iOS', 'Android'), 'web_desktop',
			device like '%android%', 'android',
			device='tv' and device_os = 'Android', 'android_tv',
			device='tv' and device_os != 'Android', 'smart_tv',
			device) as reg_device,
		if(free_days in (0,3,14,30,35,45), toString(free_days), 'other') AS free_days,
		reg_date,
		reg_source,
		reg_medium,
		reg_campaign,
		if(promo_type = 'cards', 'b2b', 'b2c') as b2b_b2c
	from datamarts.marketing_dash_distr
),

joins AS (
	SELECT
		ui.*,
		sums.* except (user_id),
		if(
			sum(total_sum_by_day) OVER (partition by user_id, paid_date) > 1, 1, 0) as payer_day_flag,
		if(
			sum(total_sum_by_week) OVER (partition by user_id, paid_week) > 1, 1, 0) as payer_week_flag,
		if(
			sum(total_sum_by_month) OVER (partition by user_id, paid_month) > 1, 1, 0) as payer_month_flag
	FROM users_info as ui
	LEFT JOIN sums ON  ui.user_id = sums.user_id
),

final_table as (
	SELECT
		* EXCEPT(paid_week, paid_month),
		IF (paid_date = '1970-01-01' and total_sum_by_day = 0, reg_date, paid_date) AS main_date,
		IF (paid_week = '1970-01-01' and total_sum_by_week = 0, toStartOfWeek(reg_date, 1), paid_date) AS main_week, 
		IF (paid_month = '1970-01-01' and total_sum_by_month = 0, toStartOfMonth(reg_date), paid_month) AS main_month
	FROM joins
	WHERE main_date >= '2023-09-01'
)

SELECT DISTINCT 
    user_id,
    reg_date,
    paid_date,
	main_date,
    main_week, 
    main_month,
    reg_device,
    free_days,
    reg_source,
    reg_medium,
    reg_campaign,
    b2b_b2c,
    platform,
    total_sum_by_day,
    total_sum_by_week,
    total_sum_by_month,
    platform_sum_by_day,
    platform_sum_by_week,
    platform_sum_by_month,
    payer_day_flag,
    payer_week_flag,
    payer_month_flag
FROM final_table
'''

execute(query,user = 'kmekhtiev')

In [ ]:
query = '''CREATE TABLE sandbox.finance_2_local ON CLUSTER 'viasat_cluster'
            (
                paid_date Date,
                subscription_id UUID,
                user_id UUID,
                platform String,
                price_currency String,
                free_days String,
                free_days_segment String,
                reg_device String,
                reg_date Date,
                reg_source String,
                reg_medium String,
                reg_campaign String,
                b2b_b2c String,
                payment Int
                )
             ENGINE = ReplicatedMergeTree('/clickhouse/tables/{shard}/finance_2_local', '{replica}')
             ORDER BY paid_date
             '''
execute(query,user = 'kmekhtiev')

In [ ]:
query = f'''CREATE TABLE sandbox.finance_2 ON CLUSTER 'viasat_cluster'
            (
                paid_date Date,
                subscription_id UUID,
                user_id UUID,
                platform String,
                price_currency String,
                free_days String,
                free_days_segment String,
                reg_device String,
                reg_date Date,
                reg_source String,
                reg_medium String,
                reg_campaign String,
                b2b_b2c String,
                payment Int
                )
             
             ENGINE = Distributed(viasat_cluster, sandbox, finance_2_local, rand())
             '''
execute(query,user = 'kmekhtiev')

In [ ]:
list_date = pd.date_range("2024-09-01", "2025-02-25", freq='D')
result = []
for date in list_date:
    date_str = date.strftime('%Y-%m-%d')
    query_delete = f" DELETE FROM sandbox.finance_2_local ON CLUSTER 'viasat_cluster' WHERE paid_date = '{date}'::date"
    execute(query_delete,user = 'kmekhtiev')
    print(f" Дата {date_str} удалена из таблицы")
    query = f''' INSERT INTO sandbox.finance_2
WITH money_table AS(SELECT 
                        paid_at::date AS paid_date,
                        user_id,
                        subscription_id,
                        platform,
                        i.price_cents AS price_cents,
                        i.price_currency AS price_currency,
                        CASE WHEN i.price_currency='USD' THEN i.price_cents*usd_value/100
                             WHEN i.price_currency='AMD' THEN i.price_cents*amd_value/100
                             ELSE i.price_cents/100
                        END bill,
                        usd_value,
                        amd_value
                        FROM  raw.viju__product_x__public__invoices i
                        LEFT JOIN raw.viju__product_x__public__subscriptions s ON s.id = i.subscription_id
                        LEFT JOIN raw.viju__product_x__public__users u ON u.id = s.user_id
                        LEFT JOIN raw.external__exchange_rate e ON e.extract_date=i.paid_at::date
                        WHERE u.user_type = 'regular'
                        AND u.vipplay = FALSE
                        AND s.state in ('normal_period','trial','canceled','grace_period')
                        AND i.paid_at::date='{date}'::date
                        AND i.state in ('success')
                        AND ((u.email NOT ILIKE '%%@test%%' AND u.email NOT ILIKE '%%@viasat%%') OR (u.email IS NULL AND u.phone_number IS NOT NULL))
                        AND i.price_cents > 100
                       ),
users_info AS (SELECT
                    user_id,
                    multiIf(
                            device='web' AND device_os IN ('iOS', 'Android'), 'web_mobile',
                            device='web' AND device_os NOT IN ('iOS', 'Android'), 'web_desktop',
                            device LIKE '%android%', 'android',
                            device='tv' AND device_os = 'Android', 'android_tv',
                            device='tv' AND device_os != 'Android', 'smart_tv',
                            device) AS reg_device,
                    free_days,        
                    IF(free_days IN (0,3,14,30,35,45), toString(free_days), 'other') AS free_days_segment,
                    reg_date,
                    reg_source,
                    reg_medium,
                    reg_campaign,
                    IF(promo_type = 'cards', 'b2b', 'b2c') as b2b_b2c
                    FROM datamarts.marketing_dash
                    )

SELECT
paid_date,
subscription_id,
m.user_id AS user_id,
platform,
price_currency,
free_days,
free_days_segment,
reg_device,
reg_date,
reg_source,
reg_medium,
reg_campaign,
b2b_b2c,
sum(bill) AS payment
FROM money_table AS m
LEFT JOIN users_info AS i ON i.user_id=m.user_id
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13
'''
    
    df_temp = execute(query,user = 'kmekhtiev')   
    result.append(df_temp)
    print(f"""Дата '{date_str}' загружена""")
    print()

In [ ]:
%%timeit
df = pd.DataFrame([('Python','6'),('Python','8'),('ML','20'),('Deeplearning','10')],columns = ['block','lessons'])
result = []
for i in range(1,len(df)+1):
    result.append(f'coursera_{i}')
    
df['number']=result

In [ ]:
%%timeit
df = pd.DataFrame([('Python','6'),('Python','8'),('ML','20'),('Deeplearning','10')],columns = ['block','lessons'])
result = []
for i in range(1,df.shape[0]+1):
    result.append(f'coursera_{i}')
    
df['number']=result

In [ ]:
x = np.arange(-100,100)
y = x**2

plt.plot(x,y)

In [ ]:
query = '''
SELECT 
paid_at::date AS paid_date,
user_id,
subscription_id,
platform,
i.price_cents AS price_cents ,
i.price_currency AS price_currency,
CASE WHEN i.price_currency='USD' THEN i.price_cents*usd_value/100
     WHEN i.price_currency='AMD' THEN i.price_cents*amd_value/100
     ELSE i.price_cents/100
END payment,
usd_value,
amd_value
FROM  product_x.invoices i
LEFT JOIN product_x.subscriptions s ON s.id = i.subscription_id
LEFT JOIN product_x.users u ON u.id = s.user_id
LEFT JOIN raw.external__exchange_rate e ON e.extract_date=i.paid_at::date
WHERE u.user_type = 'regular'
AND u.vipplay = FALSE
AND s.state in ('normal_period','trial','canceled','grace_period')
AND i.paid_at::date>='2024-01-01'
AND i.state in ('success')
AND ((u.email NOT ILIKE '%%@test%%' AND u.email NOT ILIKE '%%@viasat%%') OR (u.email IS NULL AND u.phone_number IS NOT NULL))
AND i.price_cents > 100
'''

df = execute(query,user='kmekhtiev')

In [ ]:
df['platform'].unique()

In [ ]:
df.groupby('price_currency')['price_cents'].sum()

In [ ]:
df_agg = df.groupby(['paid_date','price_currency','platform'],as_index=False).agg({'user_id':'nunique','payment':'sum'})

df_agg = df_agg[df_agg['price_currency'].isin(['USD'])]
plt.figure(figsize=(15,8))

sns.lineplot(data=df_agg,x='paid_date',y='payment',hue='price_currency')

In [ ]:
df_agg

In [ ]:
query = ''' SELECT t1.*,t2.free_days as free_days_all
FROM datamarts.finance t1
LEFT JOIN datamarts.marketing_dash AS t2 ON t1.user_id=t2.user_id
'''

df = execute(query,user='kmekhtiev')

df['paid_date'] = df['paid_date'].astype('datetime64[ns]')

In [ ]:
df_agg = df[(df['free_days']=='other')&(df['paid_date']>='2025-01-01') & (df['free_days_all'].isin([28,59,31,90,120]))].groupby(['free_days_all','paid_date'])['user_id'].nunique().reset_index()
df_agg

In [ ]:
df_agg.groupby('free_days_all')['user_id'].sum().reset_index().sort_values(by='user_id')

In [ ]:
plt.figure(figsize=(15,12),dpi=100)

sns.lineplot(data=df_agg,x='paid_date',y='user_id',hue='free_days_all',marker='o',palette='magma')

In [ ]:
df[(df['free_days_all']==45) & (df['paid_date']>='2025-01-01')].groupby('paid_date').agg({'user_id':'count','payment':'mean'})

In [ ]:
df[(df['paid_date']>='2025-02-14')].agg({'user_id':'count','payment':'mean'})

# Анализ конверсий в триалы от органики

In [23]:
query = '''SELECT 
            reg_date,
            uniq(profile_id) AS cnt_all_reg,
            uniq(CASE WHEN reg_source='none' THEN profile_id END) AS cnt_organic_reg,

            uniq(CASE WHEN created_at::date>=reg_date THEN profile_id END) AS cnt_all_trial,
            uniq(CASE WHEN created_at::date>=reg_date AND reg_source='none' THEN profile_id END) AS cnt_organic_trial,
            uniq(CASE WHEN created_at::date>=reg_date AND reg_source='none' AND free_days IN ('3','14') THEN profile_id END) AS cnt_organic_trial_general_offer,

            cnt_all_trial/cnt_all_reg AS cr_trail_all,
            cnt_organic_trial/cnt_organic_reg AS cr_trial_organic,
            cnt_organic_trial_general_offer/cnt_organic_reg AS cr_trial_organic_general_offer

            FROM datamarts.marketing_dash
            WHERE promo_type!='cards' AND reg_date between '2025-03-01' AND '2025-04-22'
            GROUP BY 1
            '''

df = execute(query,user='kmekhtiev')
df['reg_date'] = pd.to_datetime(df['reg_date'])

Время выполнения функции: 0.7041 секунд



In [25]:
df

,reg_date,cnt_all_reg,cnt_organic_reg,cnt_all_trial,cnt_organic_trial,cnt_organic_trial_general_offer,cr_trail_all,cr_trial_organic,cr_trial_organic_general_offer
0,2025-03-01,2087,1323,479,263,45,0.229516,0.198791,0.034014
1,2025-03-02,1796,1172,422,240,43,0.234967,0.204778,0.036689
2,2025-03-03,1737,979,411,204,42,0.236615,0.208376,0.042901
3,2025-03-04,1674,935,369,188,24,0.220430,0.201070,0.025668
4,2025-03-05,1698,978,369,182,29,0.217314,0.186094,0.029652
5,2025-03-06,1698,1079,338,173,27,0.199058,0.160334,0.025023
6,2025-03-07,1345,800,298,122,21,0.221561,0.152500,0.026250
7,2025-03-08,1140,693,260,136,31,0.228070,0.196248,0.044733
8,2025-03-09,1354,796,308,165,39,0.227474,0.207286,0.048995
9,2025-03-10,1251,746,283,143,30,0.226219,0.191689,0.040214
